<a href="https://colab.research.google.com/github/allarassemjonathan/tafl/blob/main/tafl_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}

In [ ]:
first_search_page = requests.get('http://aagenielsen.dk/visallespil_soeg.php?alt=1', headers=headers)

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(first_search_page.content, 'html.parser')

In [ ]:
first_body = {
    'spiltype': '16'
}
list_page = requests.post('http://aagenielsen.dk/visallespil.php', data=first_body,headers=headers)

In [ ]:
list_soup = BeautifulSoup(list_page.content, 'html.parser')

In [ ]:
all_games = list_soup.find_all('table')[1].find_all('tr')

In [ ]:
game_list_tds = [game.find_all('td')[1] for game in all_games]

In [ ]:
move_list_pages = []
for game in game_list_tds:
  inputs = game.find_all('input')
  data = {
      'spil': inputs[0]["value"],
      'pw': inputs[1]["value"]
  }
  game_req = requests.post('http://aagenielsen.dk/visspil.php',data=data,headers=headers)
  move_list_pages.append(game_req.content)

In [ ]:
move_list_soup = [BeautifulSoup(page, 'html.parser') for page in move_list_pages]

In [ ]:
move_tables = [soup.table for soup in move_list_soup]

In [ ]:
get_str = lambda x: x.string.split(' ')[0]

In [ ]:
game_move_tuples = [(list(map(get_str, table.find_all('td')[1::3])), list(map(get_str, table.find_all('td')[2::3]))) for table in move_tables]

In [ ]:
games = [{'black': moves[0][1:], 'white':moves[1][1:]} for moves in game_move_tuples]

In [ ]:
tournament_indices = []
outcomes = []
for i, row in enumerate(all_games):
  txt = row.find_all('td')[2].get_text().lower()
  if 'tournament' in txt:
    tournament_indices.append(i)
  outcome = 'white' if 'white' in txt else 'black' if 'black' in txt else 'draw' if 'draw' in txt else 'ongoing' if 'ongoing' in txt else None
  outcomes.append(outcome)
  games[i]['outcome'] = outcome
len(tournament_indices)

703

In [ ]:
tournament_games =[games[i] for i in tournament_indices]

In [ ]:
game_lens = [len(game['white']) + len(game['black']) for game in tournament_games]

In [ ]:
sum(game_lens)/len(game_lens)

54.90611664295875

In [ ]:
len(list(filter(lambda x:x['outcome']=='white',tournament_games)))/len(tournament_games)

0.5547652916073968